In [ ]:
import os
import time
import pandas as pd
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
import numpy as np

def preprocess_data(file_path):
    """
    Load and preprocess the EEG data from a given file path.
    """
 
    data = pd.read_csv(file_path)
    
   
    X = data[['chn_1', 'chn_2', 'chn_3', 'chn_4', 'marker']].values
    
 
    scaler = StandardScaler().fit(X)

    X_scaled = scaler.transform(X)
    
    # Reshape data for the model (if your model requires it)
    # This depends on the model's expected input shape.
    X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)
    
    return X_reshaped

def predict_with_model(model_path, data_path):
    """
    Load the pre-trained model and predict on new data.
    """
   
    model = load_model(model_path)
    
    # Preprocess the new data
    X_new = preprocess_data(data_path)
    
    # Predict
    predictions = model.predict(X_new)
    predictions_classes = np.argmax(predictions, axis=1)
    
    # Calculate the proportion of 'Sound Detected' predictions
    sound_detected_proportion = np.mean(predictions_classes == 1)
    
    # Decide on overall prediction based on a threshold
    if sound_detected_proportion > 0.7:  
        print("Overall Prediction: Sound Detected")
    else:
        print("Overall Prediction: No Sound Detected")


model_path = '/Users/Yizhe/Desktop/EEG_All_Data/Trained_Model.h5'
data_dir = 'H:/我的云端硬盘/EEG_Record/20240417T143230'  # The directory where new files will be created

last_epoch_processed = 0

while True:
    # Try to find the next epoch file
    epoch_filename = f'epoch_{last_epoch_processed + 1}.csv'
    full_path = os.path.join(data_dir, epoch_filename)

    if os.path.isfile(full_path):
        print(f"New file detected: {epoch_filename}")
        predict_with_model(model_path, full_path)
        last_epoch_processed += 1  # Update the counter after processing
    else:
        print(f"Waiting for file: {epoch_filename}")

    time.sleep(1)  # Check every second for a new file
